In [165]:
import pandas as pd
import numpy as np
import os

In [166]:
#pip install pandas

In [167]:
# Helper Methods

def generate_column_info(df):
    cls = []
    nullCount = []
    nonNullCount = []
    nullsPct = []
    uniqCount = []
    dataType = []
    for i,col in enumerate(df.columns):
        cls.append(col)
        nullCount.append(df[col].isnull().sum())
        nonNullCount.append(len(df)-df[col].isnull().sum())
        nullsPct.append((df[col].isnull().sum())*(100)/len(df))
        uniqCount.append(df[col].nunique())
        dataType.append(df[col].dtype)
        
    column_info = pd.DataFrame(
        {'ColumnName': cls,
         'NullCount': nullCount,
         'NonNullCount': nonNullCount,
         'NullPercent': nullsPct,
         'UniqueValueCount': uniqCount,
         'DataType':dataType
        })
    return(column_info)

## Data

In [168]:
abbr_dict = {
    'flight_Liquefied Natural Gas ImpExp Equipment.xls':'LNG_IMEX',
    'flight_Liquefied Natural Gas Storage.xls':'LNG_STOR',
    'flight_Natural Gas Local Distribution Companies.xls':'NG_LDC',
    'flight_Natural Gas Processing.xls':'NG_PROC',
    'flight_Natural Gas TransmissionCompression.xls':'NG_COMP',
    'flight_Offshore Petroleum & Natural Gas Production.xls':'PROD_OFF',
    'flight_Onshore Natural Gas Transmission Pipelines.xls':'NG_PIPE',
    'flight_Onshore Petroleum & Natural Gas Gathering & Boosting.xls':'NG_GNB',
    'flight_Onshore Petroleum & Natural Gas Production.xls':'PROD_ON',
    'flight_Other Petroleum & Natural Gas Systems.xls':'ONG_OTH',
    'flight_Underground Natural Gas Storage.xls':'NG_STOR',
}

sub_sector_dict = {
    'NG_LDC':'NaturalGasDistribution',
    'PROD_OFF':'OffshoreProduction',
    'PROD_ON':'OnshoreProduction',
}

In [169]:
#pip install pyxlsb

## Facilities List

In [170]:
#directory_path = 'C://Users//shashank.bekal//PythonWorkArea//Jnana-Research//OKC-Facilities//files'
directory_path = './/files'
all_files = os.listdir(directory_path)
xls_files = [file for file in all_files if file.endswith('.xls')]

facilities_master_df = pd.DataFrame()
for file_name in xls_files:
    file_dict = pd.read_excel(f'.//files/{file_name}',sheet_name=None,skiprows=6)
    file_df = pd.concat(file_dict, ignore_index=True)
    file_df["Abbr"] = abbr_dict[file_name]
    #print((file_name))
    facilities_master_df = pd.concat([facilities_master_df,file_df])
#print('---')
print(len(facilities_master_df))

C:\Users\shankze\AppData\Local\Temp\ipykernel_6812\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_6812\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_6812\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a 

30781


C:\Users\shankze\AppData\Local\Temp\ipykernel_6812\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)


In [171]:
required_subsectors = ['PROD_OFF','PROD_ON']

# filter by 3 required subsectors
facilities_master_df = facilities_master_df[facilities_master_df['Abbr'].isin(required_subsectors)]
facilities_master_df['sub_sector'] = facilities_master_df['Abbr'].map(sub_sector_dict)
facilities_master_df = facilities_master_df.rename(columns={'GHG QUANTITY (METRIC TONS CO2e)':'FACILITY_WIDE_GHG_QUANTITY'})
facilities_master_df = facilities_master_df.rename(columns={'GHGRP ID':'facility_id','REPORTING YEAR':'reporting_year'})
facilities_master_df = facilities_master_df.drop(['FACILITY NAME','SUBPARTS'],axis=1)

In [172]:
#facilities_master_df['sub_sector'].value_counts()

In [173]:
generate_column_info(facilities_master_df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,reporting_year,0,8055,0.000000,13,object
1,facility_id,0,8055,0.000000,1149,object
2,REPORTED ADDRESS,1645,6410,20.422098,746,object
3,LATITUDE,0,8055,0.000000,786,float64
4,LONGITUDE,0,8055,0.000000,784,float64
5,CITY NAME,0,8055,0.000000,191,object
6,COUNTY NAME,5348,2707,66.393544,166,object
7,STATE,0,8055,0.000000,22,object
8,ZIP CODE,0,8055,0.000000,268,object
9,PARENT COMPANIES,42,8013,0.521415,1284,object


## EPA Verified Facilities List

In [174]:
epa_ver_df = pd.read_excel('EPA_verified_facilities.xls',sheet_name=None,skiprows=6)
epa_ver_df = pd.concat(epa_ver_df, ignore_index=True)
epa_ver_df['REPORTING YEAR'] = epa_ver_df['REPORTING YEAR'].astype(int)
epa_ver_df['GHGRP ID'] = epa_ver_df['GHGRP ID'].astype(int)
epa_ver_df = epa_ver_df.rename(columns={'GHGRP ID':'facility_id','REPORTING YEAR':'reporting_year'})
epa_verified_unique_facilities_df = epa_ver_df[['facility_id', 'reporting_year']].drop_duplicates()
epa_verified_unique_facilities_df['EPA_verified'] = 1
epa_verified_unique_facilities_df

C:\Users\shankze\AppData\Local\Temp\ipykernel_6812\3647009640.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  epa_ver_df = pd.concat(epa_ver_df, ignore_index=True)


,facility_id,reporting_year,EPA_verified
0,1000355,2023,1
1,1014874,2023,1
2,1009238,2023,1
3,1009265,2023,1
4,1008287,2023,1
...,...,...,...
7606,1009142,2011,1
7607,1008229,2011,1
7608,1008230,2011,1
7609,1008232,2011,1


## Facilities Metadata

In [175]:
# facilities_metadata_df = pd.read_csv('rlps_ghg_emitter_facilities.csv')
# facilities_metadata_df = facilities_metadata_df[['facility_id','year','epa_verified']]
# facilities_metadata_df

In [176]:
# facilities_metadata_df.columns

In [177]:
# facilities_metadata_df[facilities_metadata_df.duplicated(subset=['facility_id','year'])]

In [178]:
# facilities_master_df = pd.merge(facilities_master_df,facilities_metadata_df,how='left',left_on=['GHGRP ID','REPORTING YEAR'],right_on=['facility_id','year'])

In [179]:
# facilities_master_df

In [180]:
# facilities_master_df.columns

In [181]:
# generate_column_info(facilities_master_df)

## Combustion Sources

In [182]:
#emissions_by_unit_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx',engine="openpyxl", sheet_name="UNIT_DATA")
c_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx', sheet_name="UNIT_DATA",skiprows=6)
c_df['Reporting Year'] = c_df['Reporting Year'].astype(int)
c_df['Facility Id'] = c_df['Facility Id'].astype(int)
c_df = c_df.rename(columns={'Facility Id':'facility_id','Facility Name':'facility_name','Reporting Year':'reporting_year'})
c_df = c_df.rename(columns={'Unit Type':'reporting_category','Unit CO2 emissions (non-biogenic) ':'total_co2_emissions','Unit Methane (CH4) emissions ':'total_ch4_emissions'})
c_df = c_df.rename(columns={'Unit Nitrous Oxide (N2O) emissions ':'total_n2o_emissions'})
c_df = c_df.drop(columns=['Unit Reporting Method','City','Primary NAICS Code','FRS Id','Industry Type (subparts)','Industry Type (sectors)','Unit Biogenic CO2 emissions (metric tons)'])
c_df = c_df.rename(columns={'Unit Maximum Rated Heat Input Capacity (mmBTU/hr)':'maximum_rated_heat_input_capacity','State':'state'})
#c_df = c_df[c_df['reporting_category'].notna()]
c_df['subpart'] = 'SubpartC'
c_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = c_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
c_df

,facility_id,facility_name,state,reporting_year,Unit Name,reporting_category,maximum_rated_heat_input_capacity,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,subpart
0,1001155,1500 South Tibbs LLC d/b/a Aurorium Indianapol...,IN,2023,GP-1,NaN,91.8,64740.5,30.50,36.356,SubpartC
1,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.00,OCS (Other combustion source),30.0,3304.7,1.50,1.788,SubpartC
2,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.01,OCS (Other combustion source),30.0,0.2,0.00,0.000,SubpartC
3,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.02,OCS (Other combustion source),30.0,175.0,0.00,0.000,SubpartC
4,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2017,CP-03.00,OCS (Other combustion source),30.0,9106.1,4.25,5.066,SubpartC
...,...,...,...,...,...,...,...,...,...,...,...
268773,1003318,ZYBACH CRYOGENIC PLANT,TX,2015,GP-Zybach,OCS (Other combustion source),32.0,33757.8,16.00,19.072,SubpartC
268774,1003318,ZYBACH CRYOGENIC PLANT,TX,2014,GP-Zybach,OCS (Other combustion source),32.0,32901.2,15.50,18.476,SubpartC
268775,1003318,ZYBACH CRYOGENIC PLANT,TX,2013,GP-Zybach,OCS (Other combustion source),32.0,42700.7,20.25,24.138,SubpartC
268776,1003318,ZYBACH CRYOGENIC PLANT,TX,2012,GP-Zybach,OCS (Other combustion source),32.0,46840.7,22.00,26.224,SubpartC


# Emission Source

In [183]:
w_df = pd.read_csv('ef_w_emissions_source_ghg.csv',encoding = 'ISO-8859-1')
print(len(w_df))
w_df['reporting_year'] = w_df['reporting_year'].astype(int)
w_df['facility_id'] = w_df['facility_id'].astype(int)
w_df = w_df.drop(columns=['bamm_desc_source_summary','bamm_indicator_source_summary','industry_segment'], axis=1)
w_df = w_df.rename(columns={'total_reported_co2_emissions':'total_co2_emissions'})
w_df = w_df.rename(columns={'total_reported_ch4_emissions':'total_ch4_emissions'})
w_df = w_df.rename(columns={'total_reported_n2o_emissions':'total_n2o_emissions'})
w_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = w_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
w_df['total_ch4_emissions'] = w_df['total_ch4_emissions'] * 25
w_df['total_n2o_emissions'] = w_df['total_n2o_emissions'] * 298
w_df['subpart'] = 'SubpartW'
w_df

503060


,basin_associated_with_facility,facility_id,facility_name,reporting_category,reporting_year,total_ch4_emissions,total_co2_emissions,total_n2o_emissions,subpart
0,NaN,1003447,UGC - DRAGON TRAIL GAS PLANT - 08-103-00036,Enhanced Oil Recovery Hydrocarbon Liquids [98....,2016,0.00,0.0,0.0,SubpartW
1,NaN,1004777,Great Lakes Gas Transmission Pipeline Shevlin ...,Completions and Workovers with Hydraulic Fract...,2023,0.00,0.0,0.0,SubpartW
2,"220 - Gulf Coast Basin (LA, TX)",1009706,"SilverBow Resources, LLC - Basin 220",Completions and Workovers without Hydraulic Fr...,2020,0.00,0.0,0.0,SubpartW
3,NaN,1012154,580 - San Juan Basin Gathering/Boosting,Dehydrators [98.236(e)],2018,94270.75,1223.8,0.0,SubpartW
4,NaN,1004849,CONROE NATURAL GAS PROCESSING PLANT,Reciprocating Compressors [98.236(p)],2015,1825.50,4.4,0.0,SubpartW
...,...,...,...,...,...,...,...,...,...
503055,NaN,1005688,STATION #7 - MOUNTAINAIR CMP STN,Natural Gas Driven Pneumatic Pumps [98.236(c)],2018,0.00,0.0,0.0,SubpartW
503056,NaN,1007441,Eunice Gas Plant,Combustion Equipment at Onshore Petroleum and ...,2021,0.00,0.0,0.0,SubpartW
503057,395 - Williston Basin,1010002,Murex Petroleum Corporation - Williston Basin 395,Completions and Workovers without Hydraulic Fr...,2019,0.00,0.0,0.0,SubpartW
503058,NaN,1012624,Repsol Pipelines LLC - Eagle Ford,Onshore Production [98.236(aa)(1)],2016,0.00,0.0,0.0,SubpartW


In [184]:
# #w_df[(w_df['reporting_year']==2022) & (w_df['facility_id']==1000435)]
# temp_df = w_df[(w_df['reporting_year']==2022) & (w_df['facility_id']==1000435)]#[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# temp_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = temp_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
# temp_df['total_ghg_emissions'] = temp_df['total_co2_emissions'] + temp_df['total_ch4_emissions'] + temp_df['total_n2o_emissions']
# temp_df[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# temp_df = temp_df[temp_df['total_ghg_emissions']>0]
# temp_df[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]

## ConCat c_df and w_df

In [185]:
df = pd.concat([c_df,w_df], ignore_index=True)

In [186]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,771838,0.000000,9564,int64
1,facility_name,0,771838,0.000000,10447,object
2,state,503060,268778,65.176889,54,object
3,reporting_year,0,771838,0.000000,14,int64
4,Unit Name,503060,268778,65.176889,21995,object
5,reporting_category,9103,762735,1.179393,117,object
6,maximum_rated_heat_input_capacity,556592,215246,72.112542,6996,float64
7,total_co2_emissions,0,771838,0.000000,181252,float64
8,total_ch4_emissions,0,771838,0.000000,55506,float64
9,total_n2o_emissions,0,771838,0.000000,15814,float64


In [187]:
# df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
# df = df[df['total_ghg_emissions']>0]

In [188]:
len(df)

771838

In [189]:
df[(df['facility_id']==1001727)&(df['reporting_year']==2023)]

,facility_id,facility_name,state,reporting_year,Unit Name,reporting_category,maximum_rated_heat_input_capacity,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,subpart,basin_associated_with_facility
27079,1001727,"Boomvang Spar, East Breaks 643 Platform",TX,2023,GP-DIE,NaN,7.52,884.1,1.00,2.086,SubpartC,NaN
27080,1001727,"Boomvang Spar, East Breaks 643 Platform",TX,2023,GP-NGE&NGT,NaN,61.42,28097.8,13.25,15.794,SubpartC,NaN
273019,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Equipment Leaks Surveys and Population Counts ...,NaN,0.0,0.00,0.000,SubpartW,NaN
276353,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Centrifugal Compressors [98.236(o)],NaN,0.0,0.00,0.000,SubpartW,NaN
322049,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Blowdown Vent Stacks [98.236(i)],NaN,0.0,0.00,0.000,SubpartW,NaN
381383,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Associated Gas Venting and Flaring [98.236(m)],NaN,0.0,0.00,0.000,SubpartW,NaN
397340,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Transmission Storage Tanks [98.236(k)],NaN,0.0,0.00,0.000,SubpartW,NaN
410006,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Natural Gas Driven Pneumatic Pumps [98.236(c)],NaN,0.0,0.00,0.000,SubpartW,NaN
410218,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Combustion Equipment at Onshore Petroleum and ...,NaN,0.0,0.00,0.000,SubpartW,NaN
426080,1001727,"Boomvang Spar, East Breaks 643 Platform",NaN,2023,NaN,Completions and Workovers without Hydraulic Fr...,NaN,0.0,0.00,0.000,SubpartW,NaN


## Rename Reporting Category

In [190]:
reporting_category_names_df = pd.read_csv('reporting_categories.csv')
df = pd.merge(df, reporting_category_names_df, how='left',on=['reporting_category'])
df = df.drop('reporting_category',axis=1)
df = df.rename(columns={'REVISED_COLUMNNAME':'reporting_category'})

## validate before substitutions

In [191]:
# facilities_in_required_subsectors = facilities_master_df['facility_id'].unique()

# validation_before_sub_df = df
# validation_before_sub_df = validation_before_sub_df[validation_before_sub_df.facility_id.isin(facilities_in_required_subsectors)]

# validation_before_sub_df['total_ghg_emissions'] = validation_before_sub_df['total_co2_emissions'] + validation_before_sub_df['total_ch4_emissions'] + validation_before_sub_df['total_n2o_emissions']
# validation_before_sub_df = validation_before_sub_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# validation_before_sub_df=validation_before_sub_df.groupby(['facility_id','reporting_year']).sum().reset_index()
# validation_before_sub_df = pd.merge(validation_before_sub_df,facilities_master_df[['facility_id','reporting_year','FACILITY_WIDE_GHG_QUANTITY']],how='left',on=['facility_id','reporting_year'])

# validation_before_sub_df = validation_before_sub_df[validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']>0]
# validation_before_sub_df['diff'] = (validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']-validation_before_sub_df['total_ghg_emissions']).abs()
# validation_before_sub_df['diff_percent_GHG']=100*validation_before_sub_df['diff']/validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']
# validation_before_sub_df.to_csv('validation_before_sub_df.csv', index=False)
# validation_before_sub_df

In [192]:
#validation_df[validation_df.diff_percent_GHG > 10]

## Concat Pneumatic Devices from detailed dataset

In [193]:
len(df[df.reporting_category == 'NG_PneumaticDevices'])

24595

In [194]:
df.columns

Index(['facility_id', 'facility_name', 'state', 'reporting_year', 'Unit Name',
       'maximum_rated_heat_input_capacity', 'total_co2_emissions',
       'total_ch4_emissions', 'total_n2o_emissions', 'subpart',
       'basin_associated_with_facility', 'reporting_category'],
      dtype='object')

In [195]:
# Drop Pneumatic Devices from original dataset
df = df[df.reporting_category != 'NG_PneumaticDevices']

# add them from this csv file
pneumatic_devices_df = pd.read_csv('ef_w_ngpneumatic_dev_units.csv')
pneumatic_devices_df = pneumatic_devices_df[['facility_id','reporting_year','facility_name','estimated_number_of_hours','pneumatic_device_type','total_count','pneum_device_type_tot_co2_emm','total_ch4_emissions']]
pneumatic_devices_df=pneumatic_devices_df.rename(columns={'pneumatic_device_type':'Unit Name','estimated_number_of_hours':'pneu_est_no_of_hours','total_count':'pneu_dev_total_count','pneum_device_type_tot_co2_emm':'total_co2_emissions'})
pneumatic_devices_df[['total_co2_emissions','total_ch4_emissions']] = pneumatic_devices_df[['total_co2_emissions','total_ch4_emissions']].fillna(0)
pneumatic_devices_df['total_ch4_emissions'] = pneumatic_devices_df['total_ch4_emissions'] * 25
pneumatic_devices_df['total_n2o_emissions'] = 0
pneumatic_devices_df['reporting_category'] = 'NG_PneumaticDevices'
pneumatic_devices_df['subpart'] = 'SubpartW'

df = pd.concat([df,pneumatic_devices_df], ignore_index=True)
df = df.drop(columns=['state'])

## Concat AssociatedGasVentingFlaring from detailed dataset

In [196]:
len(df[df.reporting_category == 'AssociatedGasVentingFlaring'])

22328

In [197]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AssociatedGasVentingFlaring']

In [198]:
# add them from this csv file
venting_flaring_df = pd.read_csv('ef_w_associated_ng_units.csv')

In [199]:
venting_flaring_df[['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions']] = venting_flaring_df[['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions']].fillna(0)
venting_flaring_df['total_co2_emissions'] =  venting_flaring_df['basin_ag_flaring_co2_emissions'] + venting_flaring_df['basin_ag_venting_co2_emissions']
venting_flaring_df['total_ch4_emissions'] =  venting_flaring_df['venting_ch4_emissions'] + venting_flaring_df['flaring_ch4_emissions']
venting_flaring_df = venting_flaring_df.drop(['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions'],axis=1)
venting_flaring_df = venting_flaring_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
venting_flaring_df[['total_n2o_emissions']] = venting_flaring_df[['total_n2o_emissions']].fillna(0)
venting_flaring_df['total_ch4_emissions'] = venting_flaring_df['total_ch4_emissions'] * 25
venting_flaring_df['total_n2o_emissions'] = venting_flaring_df['total_n2o_emissions'] * 298
venting_flaring_df['reporting_category'] = 'AssociatedGasVentingFlaring'
venting_flaring_df['subpart'] = 'SubpartW'
venting_flaring_df = venting_flaring_df.drop(['facility_name','industry_segment','wells_two_year_delay'], axis=1)
df = pd.concat([df,venting_flaring_df], ignore_index=True)

## Concat AtmosphericTanks from detailed dataset

In [200]:
len(df[df.reporting_category == 'AtmosphericTanks'])

22328

In [201]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AtmosphericTanks']

In [202]:
# add them from this csv file
athmospheric_tanks_df = pd.read_csv('ef_w_atm_stg_tanks_calc1or2.csv')

In [203]:
athmospheric_tanks_df[['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions']] = athmospheric_tanks_df[['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions']].fillna(0)
athmospheric_tanks_df[['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions']] = athmospheric_tanks_df[['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions']].fillna(0)

athmospheric_tanks_df['total_co2_emissions'] =  athmospheric_tanks_df['dump_valve_co2_emissions'] + athmospheric_tanks_df['flaring_co2_emissions'] + athmospheric_tanks_df['vapor_recovery_co2_emissions'] + athmospheric_tanks_df['venting_co2_emissions']
athmospheric_tanks_df['total_ch4_emissions'] =  athmospheric_tanks_df['dump_valve_ch4_emissions'] + athmospheric_tanks_df['flaring_ch4_emissions'] + athmospheric_tanks_df['vapor_recovery_ch4_emissions']+athmospheric_tanks_df['venting_ch4_emissions'] 
athmospheric_tanks_df = athmospheric_tanks_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
athmospheric_tanks_df[['total_n2o_emissions']] = athmospheric_tanks_df[['total_n2o_emissions']].fillna(0)
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions'],axis=1)
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions'],axis=1)
athmospheric_tanks_df['total_ch4_emissions'] = athmospheric_tanks_df['total_ch4_emissions'] * 25
athmospheric_tanks_df['total_n2o_emissions'] = athmospheric_tanks_df['total_n2o_emissions'] * 298
athmospheric_tanks_df=athmospheric_tanks_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','annual_ch4_recovered','annual_co2_recovered','atmospheric_tank_count',
                                             'average_a_p_i_gravity','average_separator_pressure','average_separator_temperature',
                                             'hrs_dump_valves_did_not_close','not_on_well_pad_tank_count','num_seps_with_stuck_dump_val',
                                             'number_of_wells_send_oil','tanks_venting_to_atm','tanks_with_flaring','tanks_with_vapor_recovery',
'total_volume_of_oil']]
athmospheric_tanks_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_df['subpart'] = 'SubpartW'

In [204]:
athmospheric_tanks_2_df = pd.read_csv('ef_w_atm_stg_tanks_calc3.csv')

athmospheric_tanks_2_df[['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared']] = athmospheric_tanks_2_df[['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared']].fillna(0)

athmospheric_tanks_2_df['total_co2_emissions'] =  athmospheric_tanks_2_df['co2_emissions_from_flare'] + athmospheric_tanks_2_df['co2_emissions_not_flared']
athmospheric_tanks_2_df['total_ch4_emissions'] =  athmospheric_tanks_2_df['ch4_emissions_from_flare'] + athmospheric_tanks_2_df['ch4_emissions_not_flared']
athmospheric_tanks_2_df = athmospheric_tanks_2_df.drop(['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared'],axis=1)
athmospheric_tanks_2_df = athmospheric_tanks_2_df.rename(columns={'n2o_emissions_from_flare':'total_n2o_emissions'})
athmospheric_tanks_2_df[['total_n2o_emissions']] = athmospheric_tanks_2_df[['total_n2o_emissions']].fillna(0)
athmospheric_tanks_2_df['total_ch4_emissions'] = athmospheric_tanks_2_df['total_ch4_emissions'] * 25
athmospheric_tanks_2_df['total_n2o_emissions'] = athmospheric_tanks_2_df['total_n2o_emissions'] * 298
athmospheric_tanks_2_df=athmospheric_tanks_2_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions',
                                                 'annual_oil_throughput','atmospheric_tank_count','fract_oil_throughput_flaring',
                                                'fract_throughput_with_vapor','tanks_with_flares','tanks_without_flares',
                                                'wells_with_seps','wells_without_seps']]
athmospheric_tanks_2_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_2_df['subpart'] = 'SubpartW'

In [205]:
df = pd.concat([df,athmospheric_tanks_df,athmospheric_tanks_2_df], ignore_index=True)
len(df)

814638

In [206]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,814638,0.000000,9564,int64
1,facility_name,73111,741527,8.974661,10447,object
2,reporting_year,0,814638,0.000000,14,int64
3,Unit Name,506920,307718,62.226412,21998,object
4,maximum_rated_heat_input_capacity,599392,215246,73.577712,6996,float64
5,total_co2_emissions,0,814638,0.000000,207055,float64
6,total_ch4_emissions,0,814638,0.000000,79052,float64
7,total_n2o_emissions,0,814638,0.000000,24956,float64
8,subpart,0,814638,0.000000,2,object
9,basin_associated_with_facility,680127,134511,83.488249,44,object


In [207]:
df[(df['facility_id']==1001727) & df['reporting_year']==2023]

,facility_id,facility_name,reporting_year,Unit Name,maximum_rated_heat_input_capacity,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,subpart,basin_associated_with_facility,...,tanks_with_flaring,tanks_with_vapor_recovery,total_volume_of_oil,annual_oil_throughput,fract_oil_throughput_flaring,fract_throughput_with_vapor,tanks_with_flares,tanks_without_flares,wells_with_seps,wells_without_seps


## Create total_ghg_emissions column

In [208]:
df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
df = df[df['total_ghg_emissions']>0]

In [209]:
len(df)

446390

In [210]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,446390,0.000000,9550,int64
1,facility_name,62592,383798,14.021819,10431,object
2,reporting_year,0,446390,0.000000,14,int64
3,Unit Name,169264,277126,37.918412,21539,object
4,maximum_rated_heat_input_capacity,241040,205350,53.997625,6937,float64
5,total_co2_emissions,0,446390,0.000000,207055,float64
6,total_ch4_emissions,0,446390,0.000000,79052,float64
7,total_n2o_emissions,0,446390,0.000000,24956,float64
8,subpart,0,446390,0.000000,2,object
9,basin_associated_with_facility,396485,49905,88.820314,44,object


In [211]:
df[(df['reporting_year']==2022) & (df['facility_id']==1000435)][['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]

,reporting_category,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions
446004,EquipmentLeaks,148.60000,40883.750,0.000,41032.35000
528241,CombustionEquipment,25656.00000,841.750,19.072,26516.82200
548113,ReciprocatingCompressors,0.70000,110.000,0.000,110.70000
563930,Dehydrators,3.50000,655.750,0.000,659.25000
617438,NG_PneumaticPumps,5.50000,1522.500,0.000,1528.00000
661048,WellVenting,3.00000,368.500,0.000,371.50000
712519,NG_PneumaticDevices,3.00000,811.750,0.000,814.75000
717056,NG_PneumaticDevices,1.90000,527.500,0.000,529.40000
729877,NG_PneumaticDevices,18.50000,5096.500,0.000,5115.00000
748196,AtmosphericTanks,0.02279,26.288,0.000,26.31079


## Filter df to only required subsectors

In [212]:
facilities_in_required_subsectors = facilities_master_df['facility_id'].unique()
print(len(df))
df = df[df.facility_id.isin(facilities_in_required_subsectors)]
print(len(df))

446390
97613


## Join Facilities Metadata

In [213]:
df = pd.merge(df,facilities_master_df,how='left',on=['facility_id','reporting_year'])


In [214]:
# remove rows for facility ids which do not have an exact match for the facilityid-reporting year combination in the facilities
#master
df = df[df['sub_sector'].notnull()]#.groupby(['facility_id','reporting_year']).count().reset_index()


In [215]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,97353,0.000000,1148,object
1,facility_name,48593,48760,49.914230,1280,object
2,reporting_year,0,97353,0.000000,13,object
3,Unit Name,84760,12593,87.064600,448,object
4,maximum_rated_heat_input_capacity,91394,5959,93.878977,760,float64
5,total_co2_emissions,0,97353,0.000000,41346,float64
6,total_ch4_emissions,0,97353,0.000000,46650,float64
7,total_n2o_emissions,0,97353,0.000000,10886,float64
8,subpart,0,97353,0.000000,2,object
9,basin_associated_with_facility,62363,34990,64.058632,44,object


## Remove Rows with Null Reporting Category

In [216]:
#df['facility_id'] = df['facility_id'].astype('int')
#df['reporting_year'] = df['reporting_year'].astype('int')
#df[(df['facility_id']==1002685) & (df['reporting_year']==2023)]
df[df['reporting_category'].isna()]

,facility_id,facility_name,reporting_year,Unit Name,maximum_rated_heat_input_capacity,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,subpart,basin_associated_with_facility,...,LONGITUDE,CITY NAME,COUNTY NAME,STATE,ZIP CODE,PARENT COMPANIES,FACILITY_WIDE_GHG_QUANTITY,Abbr,BASIN NAME/NUMBER,sub_sector
0,1002685,AC 857 A,2023,GP- Perdido AC-857,249.36,165355.9,81.25,104.598,SubpartC,NaN,...,-94.897778,Offshore,NaN,TX,0,SHELL PETROLEUM INC (100%),180279,PROD_OFF,NaN,OffshoreProduction
28,1014643,"Argos, MMS Platform ID 02481, Complex ID 2665-1",2023,GP-1,81.00,25626.6,26.00,61.984,SubpartC,NaN,...,-90.365633,Offshore,NaN,LA,0,BP AMERICA INC (60.5%); BHP MINERALS INTERNATI...,134118,PROD_OFF,NaN,OffshoreProduction
29,1014643,"Argos, MMS Platform ID 02481, Complex ID 2665-1",2023,GP-2,83.00,75662.4,35.75,42.614,SubpartC,NaN,...,-90.365633,Offshore,NaN,LA,0,BP AMERICA INC (60.5%); BHP MINERALS INTERNATI...,134118,PROD_OFF,NaN,OffshoreProduction
42,1004233,"Atlantis, MMS Platform ID 02481, Complex ID 12...",2023,CP-1,306.00,189286.7,89.25,106.386,SubpartC,NaN,...,-90.026984,Offshore,NaN,LA,0,BP AMERICA INC (56%); BHP MINERALS INTERNATION...,256468,PROD_OFF,NaN,OffshoreProduction
43,1004233,"Atlantis, MMS Platform ID 02481, Complex ID 12...",2023,GP-1,215.00,3769.2,3.75,9.238,SubpartC,NaN,...,-90.026984,Offshore,NaN,LA,0,BP AMERICA INC (56%); BHP MINERALS INTERNATION...,256468,PROD_OFF,NaN,OffshoreProduction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40949,1009391,Apache Corporation - Permian Basin 430,2015,NaN,NaN,668.9,0.00,0.000,SubpartW,430 - Permian Basin,...,-102.086520,Houston,NaN,TX,77056,APACHE CORP (100%),233428,PROD_ON,430 - Permian Basin,OnshoreProduction
41518,1013216,SEP Permian Basin 430 Production,2023,NaN,NaN,39.4,0.00,0.000,SubpartW,430 - Permian Basin,...,-96.823258,Dallas,NaN,TX,75240,SCOUT ENERGY MANAGEMENT LLC (100%),39621,PROD_ON,430 - Permian Basin,OnshoreProduction
41562,1014413,Contango Basin 530,2023,NaN,NaN,0.1,0.00,0.000,SubpartW,530 - Wind River Basin,...,-95.364292,Houston,NaN,TX,77002,CONTANGO RESOURCES LLC (100%),32673,PROD_ON,530 - Wind River Basin,OnshoreProduction
42113,1008885,"Tellus Operating Group, LLC - MS Operations",2023,NaN,NaN,689.9,0.00,0.000,SubpartW,210 - Mid-Gulf Coast Basin,...,-90.153930,Ridgeland,NaN,MS,39157,TELLUS ENERGY GROUP LLC (100%),19532,PROD_ON,210 - Mid-Gulf Coast Basin,OnshoreProduction


In [217]:
df = df[df['reporting_category'].notnull()]

In [218]:
#df[df['reporting_category'].isna()]

## Filter rows based on EPA verified facility_id reporting_year list

In [219]:
#epa_verified_unique_facilities_df['EPA verified'] = 1
df = pd.merge(df,epa_verified_unique_facilities_df,how='left',on=['facility_id','reporting_year'])


In [220]:
#df = df[df['GHG FACILITY_WIDE_GHG_QUANTITY']>0]

## Validate GHG Quantity for facility and add diff_percent_GHG to df

In [221]:

validation_df = df
validation_df = validation_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
validation_df=validation_df.groupby(['facility_id','reporting_year']).sum().reset_index()
validation_df = pd.merge(validation_df,facilities_master_df[['facility_id','reporting_year','FACILITY_WIDE_GHG_QUANTITY']],how='left',on=['facility_id','reporting_year'])
validation_df['diff'] = (validation_df['FACILITY_WIDE_GHG_QUANTITY']-validation_df['total_ghg_emissions']).abs()
validation_df['diff_percent_GHG']=100*validation_df['diff']/validation_df['FACILITY_WIDE_GHG_QUANTITY']
validation_df.to_csv('output_validation.csv', index=False)
validation_df = validation_df[['facility_id','reporting_year','diff_percent_GHG']]
#validation_df[validation_df.diff_percent_GHG > 10]

In [222]:
df = pd.merge(df,validation_df,how='left',on=['facility_id','reporting_year'])

## Cleanup

In [223]:
#generate_column_info(df)

In [224]:
df=df.drop(['sub_basin_identifier','table_desc','table_num','average_gas_to_oil_ratio','average_a_p_i_gravity','average_separator_pressure','average_separator_temperature','fract_oil_throughput_flaring','fract_throughput_with_vapor','wells_with_seps','wells_without_seps','Abbr'],axis=1)

In [225]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,96936,0.000000,1148,object
1,facility_name,48593,48343,50.128951,1280,object
2,reporting_year,0,96936,0.000000,13,object
3,Unit Name,84519,12417,87.190517,447,object
4,maximum_rated_heat_input_capacity,91153,5783,94.034208,756,float64
5,total_co2_emissions,0,96936,0.000000,41175,float64
6,total_ch4_emissions,0,96936,0.000000,46642,float64
7,total_n2o_emissions,0,96936,0.000000,10884,float64
8,subpart,0,96936,0.000000,2,object
9,basin_associated_with_facility,62187,34749,64.152637,44,object


## Calculate facility totals

In [226]:
#df.columns

In [227]:
fac_total_df = df[['facility_id','reporting_year','maximum_rated_heat_input_capacity','pneu_dev_total_count','number_of_wells_flaring','number_of_wells_venting', 'volume_of_gas_sent_to_sales',
       'volume_of_oil_produced', 'annual_ch4_recovered',
       'annual_co2_recovered', 'atmospheric_tank_count',
       'hrs_dump_valves_did_not_close', 'not_on_well_pad_tank_count',
       'num_seps_with_stuck_dump_val', 'number_of_wells_send_oil',
       'tanks_venting_to_atm', 'tanks_with_flaring',
       'tanks_with_vapor_recovery', 'total_volume_of_oil',
       'annual_oil_throughput', 'tanks_with_flares', 'tanks_without_flares']]

In [228]:
fac_total_df = fac_total_df.groupby(['facility_id','reporting_year']).sum().reset_index()
# rename facility total columns
fac_total_df = fac_total_df.rename(columns=lambda x: x if x in fac_total_df.columns[:2] else x + "_facility_total")
df = pd.merge(df,fac_total_df,how='left',on=['facility_id','reporting_year'])

In [229]:
fac_avg_df = df[['facility_id','reporting_year','pneu_est_no_of_hours']]
fac_avg_df = fac_avg_df[fac_avg_df['pneu_est_no_of_hours'].notnull()]
fac_avg_df = fac_avg_df[fac_avg_df['pneu_est_no_of_hours']>0]
fac_avg_df = fac_avg_df.groupby(['facility_id','reporting_year']).mean().reset_index()
fac_avg_df = fac_avg_df.rename(columns={'pneu_est_no_of_hours':'pneu_est_no_of_hours_facility_mean'})
df = pd.merge(df,fac_avg_df,how='left',on=['facility_id','reporting_year'])

In [230]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,96936,0.000000,1148,object
1,facility_name,48593,48343,50.128951,1280,object
2,reporting_year,0,96936,0.000000,13,object
3,Unit Name,84519,12417,87.190517,447,object
4,maximum_rated_heat_input_capacity,91153,5783,94.034208,756,float64
...,...,...,...,...,...,...
62,total_volume_of_oil_facility_total,0,96936,0.000000,3250,float64
63,annual_oil_throughput_facility_total,0,96936,0.000000,1,float64
64,tanks_with_flares_facility_total,0,96936,0.000000,435,float64
65,tanks_without_flares_facility_total,0,96936,0.000000,857,float64


In [231]:
# Use average of non-zero for pneu_est_no_of_hours

In [232]:
df.to_csv('output.csv', index=False)

In [233]:
#AssociatedGasVentingFlaring

# AtmosphericTanks